In [ ]:
# Installing Theano
import sys
!{sys.executable} -m pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git --user 

# Installing Tensorflow
import sys
!{sys.executable} -m pip install --upgrade tensorflow --user

# Installing Keras
import sys
!{sys.executable} -m pip install --upgrade keras --user 

# Install other packages
import sys
!{sys.executable} -m  pip install --upgrade pip pandas numpy matplotlib sklearn --user

In [1]:
# Artificial Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Part 2 - Now let's make the ANN!

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 11))
# classifier.add(Dropout(rate = 0.1))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
# classifier.add(Dropout(rate = 0.1))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)
print(new_prediction)



/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
Using TensorFlow backend.


Epoch 1/100
8000/8000 [==============================] - 2s 256us/step - loss: 0.4813 - accuracy: 0.8055
Epoch 2/100
8000/8000 [==============================] - 2s 195us/step - loss: 0.4340 - accuracy: 0.8092
Epoch 3/100
8000/8000 [==============================] - 2s 194us/step - loss: 0.4329 - accuracy: 0.8104
Epoch 4/100
8000/8000 [==============================] - 2s 195us/step - loss: 0.4324 - accuracy: 0.8127
Epoch 5/100
8000/8000 [==============================] - 2s 199us/step - loss: 0.4307 - accuracy: 0.8110
Epoch 6/100
8000/8000 [==============================] - 2s 190us/step - loss: 0.4275 - accuracy: 0.8123
Epoch 7/100
8000/8000 [==============================] - 1s 185us/step - loss: 0.4204 - accuracy: 0.8181
Epoch 8/100
8000/8000 [==============================] - 2s 193us/step - loss: 0.4093 - accuracy: 0.8251
Epoch 9/100
8000/8000 [==============================] - 2s 197us/step - loss: 0.3948 - accuracy: 0.8344
Epoch 10/100
8000/8000 [==============================]

In [2]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Part 4 - Evaluating, Improving and Tuning the ANN

# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
#from keras.layers import Dropout
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()



[[1525   70]
 [ 201  204]]
Epoch 1/10
7200/7200 [==============================] - 2s 250us/step - loss: 0.4845 - accuracy: 0.8072
Epoch 2/10
7200/7200 [==============================] - 2s 213us/step - loss: 0.4337 - accuracy: 0.8100
Epoch 3/10
7200/7200 [==============================] - 2s 229us/step - loss: 0.4326 - accuracy: 0.8124
Epoch 4/10
7200/7200 [==============================] - 2s 219us/step - loss: 0.4319 - accuracy: 0.81290s - loss: 0.4331 - accura
Epoch 5/10
7200/7200 [==============================] - 2s 228us/step - loss: 0.4309 - accuracy: 0.8128
Epoch 6/10
7200/7200 [==============================] - 2s 217us/step - loss: 0.4274 - accuracy: 0.8142
Epoch 7/10
7200/7200 [==============================] - 2s 216us/step - loss: 0.4195 - accuracy: 0.8154
Epoch 8/10
7200/7200 [==============================] - 2s 221us/step - loss: 0.4047 - accuracy: 0.8242
Epoch 9/10
7200/7200 [==============================] - 1s 207us/step - loss: 0.3858 - accuracy: 0.8343
Epoch 10/10

In [ ]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {'batch_size': [25, 32],
              'epochs': [5, 10],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_


Epoch 1/5
7200/7200 [==============================] - 1s 146us/step - loss: 0.5517 - accuracy: 0.8024
Epoch 2/5
7200/7200 [==============================] - 1s 91us/step - loss: 0.4379 - accuracy: 0.8112
Epoch 3/5
7200/7200 [==============================] - 1s 93us/step - loss: 0.4372 - accuracy: 0.8117
Epoch 4/5
7200/7200 [==============================] - 1s 102us/step - loss: 0.4367 - accuracy: 0.8135
Epoch 5/5
7200/7200 [==============================] - 1s 101us/step - loss: 0.4330 - accuracy: 0.8149
Epoch 1/5
7200/7200 [==============================] - 1s 139us/step - loss: 0.5503 - accuracy: 0.8021
Epoch 2/5
7200/7200 [==============================] - 1s 93us/step - loss: 0.4381 - accuracy: 0.8106
Epoch 3/5
7200/7200 [==============================] - 1s 87us/step - loss: 0.4342 - accuracy: 0.8118
Epoch 4/5
7200/7200 [==============================] - 1s 95us/step - loss: 0.4352 - accuracy: 0.8149
Epoch 5/5
7200/7200 [==============================] - 1s 97us/step - loss: 0.